In [2]:
%matplotlib qt5

# handling XRF HDF5 data
import h5py 
import hyperspy.api as hs

#handling data
import pandas as pd
import numpy as np

# data vis
import matplotlib.pyplot as plt
import seaborn as sns

#import glob
#import scipy

# data processing, dimension reduction and clustering
from sklearn.decomposition import PCA, FactorAnalysis
import sklearn as skl
from sklearn.pipeline import Pipeline
from skcmeans.algorithms import Probabilistic, GustafsonKesselMixin

#from skcmeans.algorithms import Probabilistic, GustafsonKesselMixin
#from skcmeans.algorithms import Probabilistic
#from skcmeans.algorithms import Probabilistic, GustafsonKesselMixin
# from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

# Convert XRF data to Hyperspy standard

In [4]:
!cd '/Users/user/Documents/Projects/active/working/XRF_machine_learning/data'
!ls '/Users/user/Documents/Projects/active/working/XRF_machine_learning/data'
!pwd

ISE_500sqaures_A21-016_Map1_001.h5
ISE_500sqaures_A21_054_botom_right_map_center_001.h5
map.hspy
map1.hspy
map<class 'map'>.hspy
/Users/user/Documents/github/melt_maps


In [6]:
data = '/Users/user/Documents/Projects/active/working/XRF_machine_learning/data/ISE_500sqaures_A21_054_botom_right_map_center_001.h5'
save_path = '/Users/user/Documents/Projects/active/working/XRF_machine_learning/data/'

In [7]:
def print_and_list_h5_objects(obj, level=0, path='', dataset_list=[]):
    """
   checks whether the input object is a file or group, prints its name/type 
   accordingly, and then goes through its subgroups and datasets, printing 
   their names/types and adding the dataset paths to the list 'dataset_list'.
   
   Returns
    -------
    prints the structure of the HDF5 file and returns a list of dataset paths
    
   Note
    -----
    This function makes it easier to look through the HDF5 blackbox and call 
    datasets.
    """
    if isinstance(obj, h5py.File):
        print("File name: {}".format(obj.filename))
        obj = obj['/']
    elif isinstance(obj, h5py.Group):
        print("Group: {}".format(obj.name))

    for key, val in obj.items():
        new_path = path + '/' + key if path else key
        if isinstance(val, h5py.Group):
            print("  " * level + "Group: {}".format(key))
            print_and_list_h5_objects(val, level + 1, new_path, dataset_list)
        else:
            print("  " * level + "Dataset: {}".format(key))
            dataset_list.append(new_path)
    
    return dataset_list

# USE EXAMPLE
# with h5py.File(data,'r') as h5:
#    datasets = print_and_list_h5_objects(h5)

In [8]:
# prints the h5 file structure
with h5py.File(data,'r') as h5:
    datasets = print_and_list_h5_objects(h5)

File name: /Users/user/Documents/Projects/active/working/XRF_machine_learning/data/ISE_500sqaures_A21_054_botom_right_map_center_001.h5
Group: xrmmap
Group: /xrmmap
  Group: areas
Group: /xrmmap/areas
    Dataset: A21-054_Br_Xanes_spot
    Dataset: A21-054_Br_Xanes_spot_2
    Dataset: A21-054_I_Xanes_spot
    Dataset: A21-054_I_Xanes_spot_2
    Dataset: area_003
    Dataset: area_004
  Group: config
Group: /xrmmap/config
    Group: environ
Group: /xrmmap/config/environ
      Dataset: address
      Dataset: name
      Dataset: value
    Group: general
Group: /xrmmap/config/general
      Dataset: basedir
      Dataset: envfile
    Group: mca_calib
Group: /xrmmap/config/mca_calib
      Dataset: offset
      Dataset: quad
      Dataset: slope
    Group: mca_settings
Group: /xrmmap/config/mca_settings
    Group: motor_controller
Group: /xrmmap/config/motor_controller
      Dataset: group
      Dataset: host
      Dataset: mode
      Dataset: passwd
      Dataset: positioners
      Dataset: 

### Extracts congiuration settings of the run
needed to correctly scale axes and the scale channels to KeV

In [9]:
# extracts the congiuration settings of the scan and beamline
with h5py.File(data, 'r') as h5:
    dataset_df = pd.DataFrame(h5['/xrmmap/config/environ/name'][:])
    value_df = pd.DataFrame(h5['/xrmmap/config/environ/value'][:])
    # Merges configuration name and value dataframes
    values_df = pd.concat([dataset_df, value_df], axis=1)
    # Decodes byte strings to regular strings
    values_df = values_df.apply(lambda x: x.str.decode('utf-8') if isinstance(x[0], bytes) else x, axis=0)
print(values_df)
# Save dataframe to CSV file
#values_df.to_csv('values_df.csv', index=False)

                                     0         0
0                 Experiment.User_Name      Ezad
1           Experiment.Proposal_Number     78722
2        Experiment.Beam_Size__Nominal       2um
3     Experiment.Monochromator_Crystal   Si(111)
4   Experiment.Double_H_Mirror_Stripes   rhodium
..                                 ...       ...
60                 SampleStage.CoarseY  177.9370
61                   SampleStage.FineX  -0.25500
62                   SampleStage.FineY         0
63                   SampleStage.Theta   90.0000
64                    Undulator.Energy    13.717

[65 rows x 2 columns]


## Axis management and scaling

these are needed to scale the data and pixelscorrectly etc.

In [10]:
# extracts xrf map and channels from the HDF5 file
with h5py.File(data, 'r') as h5:
    xrf_data = h5['/xrmmap/mcasum/counts'][:]
    shape = h5['/xrmmap/mcasum/counts'].shape
print(shape)

(201, 201, 4096)


In [11]:
# converts xrf map to hyperspy and scales data

def convert_xrf_to_hyperspy(xrf_data):
    # converts xrf map to hyperspy
    xrf_map = hs.signals.Signal1D(xrf_data)

    # scales the x, y and theta of the hyperspy data
    xrf_map.axes_manager.navigation_axes[0].name = 'x'
    xrf_map.axes_manager.navigation_axes[0].units = 'mm'
    xrf_map.axes_manager.navigation_axes[0].scale = 0.025

    xrf_map.axes_manager.navigation_axes[1].name = 'y'
    xrf_map.axes_manager.navigation_axes[1].units = 'mm'
    xrf_map.axes_manager.navigation_axes[1].scale = 0.025

    xrf_map.axes_manager.signal_axes[0].name = 'Energy'
    xrf_map.axes_manager.signal_axes[0].units = 'KeV'
    xrf_map.axes_manager.signal_axes[0].scale = 13.717/4096
    
    return xrf_map

In [12]:
xrf_map = convert_xrf_to_hyperspy(xrf_data)
print(xrf_map.axes_manager)

<Axes manager, axes: (201, 201|4096)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    201 |      0 |       0 |   0.025 |     mm 
               y |    201 |      0 |       0 |   0.025 |     mm 
---------------- | ------ | ------ | ------- | ------- | ------ 
          Energy |   4096 |      0 |       0 |  0.0033 |    KeV 


In [13]:
xrf_map.save(save_path+'map.hspy'.format(map))

Overwrite '/Users/user/Documents/Projects/active/working/XRF_machine_learning/data/map.hspy' (y/n)?
y


# Dimension reduction
can start from here once the data has been pulled from the hdf5 file

In [14]:
xrf_stack = hs.load(save_path + 'map.hspy', stack = True)

[########################################] | 100% Completed | 115.66 ms


In [16]:
print(xrf_stack.axes_manager)

<Axes manager, axes: (201, 201|4096)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    201 |      0 |       0 |   0.025 |     mm 
               y |    201 |      0 |       0 |   0.025 |     mm 
---------------- | ------ | ------ | ------- | ------- | ------ 
          Energy |   4096 |      0 |       0 |  0.0033 |    KeV 


## BIC function
from Francesco

In [13]:
def bic_elbow(signal, decomposition_method='PCA', n_components_range=np.arange(2, 11)):
    """
    Perform a BIC elbow test to find the optimal number of components for a signal decomposition method.
    Parameters
    ----------
    signal : `hyperspy.signals.Signal2D` or `hyperspy.signals.SignalND`
        The hyperspy signal to perform the BIC elbow test on.
    decomposition_method : str, optional
        The signal decomposition method to use. Default is 'PCA'.
    n_components_range : array-like, optional
        The range of possible numbers of components to fit the signal model for.
        Default is np.arange(2, 11).
    Returns
    -------
    optimal_n_components : int
        The optimal number of components that minimizes the BIC score.
    Notes
    -----
    This function uses the Bayesian Information Criterion (BIC) to select the optimal number of components
    for the given signal decomposition method. The BIC score is computed for each number of components
    in the range, and the elbow point in the BIC curve is identified as the optimal number of components.
    """
    bic_scores = []
    for n_components in n_components_range:
        # Fit the signal model
        if decomposition_method == 'PCA':
            model = signal.decomposition.PCA(n_components=n_components)
        elif decomposition_method == 'NMF':
            model = signal.decomposition.nmf(n_components=n_components)
        elif decomposition_method == 'ICA':
            model = signal.decomposition.ica(n_components=n_components)
        else:
            raise ValueError(f"Unknown decomposition method: {decomposition_method}")

        # Compute the log-likelihood and number of model parameters
        log_likelihood = model.score(signal)
        n_samples, n_features = signal.data.shape
        n_params = n_components * (n_features + 1)

        # Compute the BIC score
        bic = -2 * log_likelihood + n_params * np.log(n_samples)
        bic_scores.append(bic)

    # Plot the BIC scores as a function of the number of components
    plt.plot(n_components_range, bic_scores, 'o-')
    plt.xlabel('Number of components')
    plt.ylabel('BIC score')
    plt.title(f'BIC elbow test for {decomposition_method} decomposition')
    plt.show()

    # Find the elbow point
    diff = np.diff(bic_scores)
    elbow_index = np.argmax(diff) + 1
    optimal_n_components = n_components_range[elbow_index]
    print(f'The optimal number of components is {optimal_n_components}')

    return optimal_n_components

## PCA

In [18]:
PCA = Pipeline([("pca", PCA(n_components=40, svd_solver='full'))])
xrf_decomp_pca = xrf_stack.decomposition(normalize_poissonian_noise = True, algorithm = PCA, 
                        return_info = True, output_dimension = 40)

Decomposition info:
  normalize_poissonian_noise=True
  algorithm=Pipeline(steps=[('pca', PCA(n_components=40, svd_solver='full'))])
  output_dimension=40
  centre=None
scikit-learn estimator:
Pipeline(steps=[('pca', PCA(n_components=40, svd_solver='full'))])


In [19]:
xrf_stack.plot_cumulative_explained_variance_ratio()
xrf_stack.plot_explained_variance_ratio(log=True, vline=True)

<Axes: title={'center': 'data\nPCA Scree Plot'}, xlabel='Principal component index', ylabel='Proportion of variance'>

In [20]:
sig_components = xrf_stack.estimate_elbow_position() + 1

In [22]:
xrf_decomp_pca = xrf_stack.decomposition(normalize_poissonian_noise = True, algorithm = PCA, 
                        return_info = True, output_dimension = sig_components)

Decomposition info:
  normalize_poissonian_noise=True
  algorithm=Pipeline(steps=[('pca', PCA(n_components=40, svd_solver='full'))])
  output_dimension=7
  centre=None
scikit-learn estimator:
Pipeline(steps=[('pca', PCA(n_components=40, svd_solver='full'))])


In [75]:
pca_loadings = xrf_stack.get_decomposition_loadings()
pca_loadings.shape

AttributeError: 'BaseSignal' object has no attribute 'shape'

In [74]:
pca_loadings_array = pca_loadings.data

NameError: name 'pca_loading_array' is not defined

In [57]:
sig_loadings = pca_loadings_array[:, :sig_components]
np.save('sig_loadings.npy', sig_loadings)
print(sig_loadings.shape)

(7, 7, 201)


In [29]:
# number of rows and columns for subplot grid
num_pcs = sig_components - 1
num_cols = 3
if num_pcs > 0:
    num_rows = int(np.ceil(num_pcs / num_cols))
else:
    num_rows = 1
    num_cols = 1

# figure; grid of subplots
fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, 3*num_rows))

# plots the data on subplots
for i in range(num_pcs):
    row = i // num_cols
    col = i % num_cols
    axs[row, col].scatter(sig_loadings[:, 0], sig_loadings[:, i+1], s=5)
    axs[row, col].set_xlabel("PC 1")
    axs[row, col].set_ylabel("PC {}".format(i+2))

# removes subplots without data
for i in range(num_pcs, num_rows*num_cols):
    axs.flat[i].remove()

plt.tight_layout()

# saves figure
plt.savefig('pca.png', dpi=100)
plt.show()

In [ ]:
pca_loads, ydim, xdim = pca_load.data.shape

fact_load_vect= pd.DataFrame((fa_load.data.reshape(fa_facts, ydim*xdim).T), columns = ['Factor 1','Factor 2','Factor 3','Factor 4', 'Factor 5'])

In [71]:
# Reshape into 2D
n_samples = sig_loadings.shape[0] * sig_loadings.shape[1]
n_features = sig_loadings.shape[2]
sig_loadings_2d = sig_loadings.reshape(n_samples, n_features)
print(sig_loadings_2d.shape)

(49, 201)


## FA

In [73]:
factors = 5 # we need a way to determine factors
pipeline = Pipeline([("fa", FactorAnalysis(n_components = factors))])
xrf_decomp = xrf_stack.decomposition(normalize_poissonian_noise = True, 
                                      algorithm = pipeline, 
                                      return_info = True, 
                                      output_dimension = 5)

Decomposition info:
  normalize_poissonian_noise=True
  algorithm=Pipeline(steps=[('fa', FactorAnalysis(n_components=5))])
  output_dimension=5
  centre=None
scikit-learn estimator:
Pipeline(steps=[('fa', FactorAnalysis(n_components=5))])


In [92]:
xrf_stack.plot_decomposition_results()

In [93]:
fa_load = xrf_stack.get_decomposition_loadings()
#fa_load.save('snv_eds_analysis/plage_TD_eds_FA_4_load')

fa_fact = xrf_stack.get_decomposition_factors()
#fa_fact.set_elements(elements)

#fa_fact.save('snv_eds_analysis/plage_TD_eds_FA_4_fact')

In [94]:
fa_load_array = fa_load.data
fa_fact_array = fa_fact.data

In [95]:
fa_load_array.shape, fa_fact_array.shape

((3, 201, 201), (3, 4096))

In [ ]:
fa_facts, ydim, xdim = fa_load.data.shape

fact_load_vect = pd.DataFrame((fa_load.data.reshape(fa_facts, ydim * xdim).T), columns = ['Factor 1','Factor 2','Factor 3','Factor 4'])

In [ ]:
fact_load_vect

In [ ]:
fa_facts, ydim,xdim = fa_load.data.shape
fa_vect = fa_load.data.reshape(fa_facts,ydim*xdim).transpose()
fa_vect.shape

In [ ]:
loadings_to_cluster = fa_vect[:,:fa_facts]
loadings_to_cluster.shape

In [ ]:
# Calculate the number of rows and columns for the plot grid
if len(fa_load_array.shape) == 0:
    num_fac = 1
else:
    num_fac = fa_load_array.shape[0] - 1
num_cols = 3
num_rows = int(np.ceil(num_fac / num_cols))

# Create a figure and a grid of subplots
fig, axs = plt.subplots(num_rows, num_cols, figsize=(10, 3*num_rows))

# plots the data on subplots
for i in range(num_fac):
    row = i // num_cols
    col = i % num_cols
    axs[row, col].scatter(fa_load_array[:, 0], fa_load_array[:, i+1], s=5)
    axs[row, col].set_xlabel("Factor loading 1")
    axs[row, col].set_ylabel("Factor loading {}".format(i+2))

# removes subplots without data
for i in range(num_fac, num_rows*num_cols):
    axs.flat[i].remove()

plt.tight_layout()

# saves figure
plt.savefig('fa.png', dpi=100)
plt.show()

In [101]:
# number of rows and columns for subplot grid
num_fac = fa_fact_array.shape[0] - 1
num_cols = 3
num_rows = int(np.ceil(num_fac / num_cols))

# figure; grid of subplots
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 5*num_rows))

# plots the data on subplots
for i in range(num_fac):
    row = i // num_cols
    col = i % num_cols
    axs[row, col].scatter(fa_fact_array[:, 0], fa_fact_array[:, i+1])
    axs[row, col].set_xlabel("Factor 1")
    axs[row, col].set_ylabel("Factor {}".format(i+2))

# removes subplots without data
for i in range(num_fac, num_rows*num_cols):
    axs.flat[i].remove()

plt.tight_layout()

# saves figure
plt.savefig('fa.png', dpi=100)
plt.show()

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

## ICA on PCA loadings
blind source separation (BSS)

In [ ]:
# uses PCA results for ICA


## Clustering
using fuzzy c-means

In [64]:
class GKProbabilistic(Probabilistic, GustafsonKesselMixin):
    pass

In [66]:
num_clus = sig_components
clusterer = GKProbabilistic(n_clusters=num_clus, n_init=20)

In [72]:
clusterer.fit(sig_loadings_2d)

LinAlgError: Singular matrix

In [63]:
pgk = PGK(n_clusters =num_clus, n_init=20).fit(sig_loadings_2d)
# Process results for visualisation
print(pgk.memberships_)
labels_ = np.argmax(pgk.memberships_, axis=1)
memberships_ = pgk.memberships_[range(len(pgk.memberships_)), labels_] 

labels = labels_.reshape([ydim,xdim])
labels=hs.signals.Signal2D(labels)
labels.plot(cmap='tab10')

mems=pgk.memberships_.reshape(ydim,xdim,num_clus)
print(mems.shape)

mem_maps=hs.signals.Signal2D(mems)
mem_maps=mem_maps.transpose(signal_axes=(2,0))
mem_maps.change_dtype('float32')
mem_maps.plot(cmap='viridis')

LinAlgError: Singular matrix

In [68]:
# run c-means clustering for each value of k
wcss = []
for k in k_range:
    cmeans = Probabilistic(n_clusters=k, max_iterations=1000, random_state=0)
    cmeans.fit(sig_loadings_2d)
    u, u0, d, jm, p, fpc = cmeans.cmeans_predict(sig_loadings_2d, return_probabilities=True)
    wcss.append(np.sum((sig_loadings_2d - np.dot(u, cmeans.cluster_centers_))**2))

NameError: name 'k_range' is not defined

In [ ]:
# plot the WCSS as a function of k
plt.plot(k_range, wcss)
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.title('Elbow method for optimal number of clusters')
plt.show()

In [ ]:
num_clus=bic
pgk = PGK(n_clusters =num_clus, n_init=10).fit(loadings_to_cluster)
# Process results for visualisation
print(pgk.memberships_)
labels_ = np.argmax(pgk.memberships_, axis=1)
memberships_ = pgk.memberships_[range(len(pgk.memberships_)), labels_] 

labels = labels_.reshape([ydim,xdim])
labels=hs.signals.Signal2D(labels)
labels.plot(cmap='tab10')

mems=pgk.memberships_.reshape(ydim,xdim,num_clus)
print(mems.shape)

mem_maps=hs.signals.Signal2D(mems)
mem_maps=mem_maps.transpose(signal_axes=(2,0))
mem_maps.change_dtype('float32')
mem_maps.plot(cmap='viridis')

In [ ]:
num_clus=bic
pgk = PGK(n_clusters =num_clus, n_init=10).fit(loadings_to_cluster)
# Process results for visualisation
print(pgk.memberships_)
labels_ = np.argmax(pgk.memberships_, axis=1)
memberships_ = pgk.memberships_[range(len(pgk.memberships_)), labels_] 

labels = labels_.reshape([ydim,xdim])
labels=hs.signals.Signal2D(labels)
labels.plot(cmap='tab10')

mems=pgk.memberships_.reshape(ydim,xdim,num_clus)
print(mems.shape)

mem_maps=hs.signals.Signal2D(mems)
mem_maps=mem_maps.transpose(signal_axes=(2,0))
mem_maps.change_dtype('float32')
mem_maps.plot(cmap='viridis')

# Signal sums from regions of interest
selected from the dimension reduction and the clustering

# working...

## Dimensions of the map
also sets save paths...

In [210]:
xrf_map.change_dtype('float32')
#xrf_map.save(save_path+'map1')

xrf_stack = xrf_map

In [211]:
xrf_stack

<Signal1D, title: , dimensions: (201, 201|4096)>

In [212]:
xrf_stack.plot()

In [83]:
xrf_stack.change_dtype('float32')


xrf_stack.save(save_path+'at16_map2_002_mapped_crop')

Overwrite '/Users/joshuashea/melt_mapsat16_map2_002_mapped_crop.hspy' (y/n)?
y


In [84]:
plt.close('all')

In [213]:

xrf_stack.decomposition(normalize_poissonian_noise=False, algorithm="sklearn_pca", output_dimension=20)

xrf_stack.plot_explained_variance_ratio(log=True, vline=True)

xrf_stack.plot_decomposition_results()

Decomposition info:
  normalize_poissonian_noise=False
  algorithm=sklearn_pca
  output_dimension=20
  centre=None
scikit-learn estimator:
PCA(n_components=20)


In [128]:
xrf_stack.plot_cumulative_explained_variance_ratio()

<AxesSubplot:xlabel='Principal component', ylabel='Cumulative explained variance ratio'>

**note** iterative cropping above showed that most data/ analysis was better when cropping down to the shape (157, 63|2001)

In [118]:
xrf_stack.save(save_path+'lisheen_low_res_map_data_Crop')

# New PCA

In [ ]:
xrf_stack = xrf_map

In [ ]:
xrf_stack.decomposition(normalize_poissonian_noise=True, algorithm='PCA', output_dimension=15)
xrf_stack.plot_cumulative_explained_variance_ratio()
xrf_stack.plot_explained_variance_ratio(log=True, vline=True)
xrf_stack.plot_decomposition_results()

In [32]:
def bic_elbow(signal, decomposition_method='sklearn_pca', n_components_range=np.arange(2, 11)):
    """
    Perform a BIC elbow test to find the optimal number of components for a signal decomposition method.
    Parameters
    ----------
    signal : `hyperspy.signals.Signal2D` or `hyperspy.signals.SignalND`
        The hyperspy signal to perform the BIC elbow test on.
    decomposition_method : str, optional
        The signal decomposition method to use. Default is 'PCA'.
    n_components_range : array-like, optional
        The range of possible numbers of components to fit the signal model for.
        Default is np.arange(2, 11).
    Returns
    -------
    optimal_n_components : int
        The optimal number of components that minimizes the BIC score.
    Notes
    -----
    This function uses the Bayesian Information Criterion (BIC) to select the optimal number of components
    for the given signal decomposition method. The BIC score is computed for each number of components
    in the range, and the elbow point in the BIC curve is identified as the optimal number of components.
    """
    bic_scores = []
    for n_components in n_components_range:
        # Fit the signal model
        if decomposition_method == 'sklearn_pca':
            model = signal.decomposition.PCA(n_components=n_components)
        elif decomposition_method == 'NMF':
            model = signal.decomposition.nmf(n_components=n_components)
        elif decomposition_method == 'ICA':
            model = signal.decomposition.ica(n_components=n_components)
        else:
            raise ValueError(f"Unknown decomposition method: {decomposition_method}")

        # Compute the log-likelihood and number of model parameters
        log_likelihood = model.score(signal)
        n_samples, n_features = signal.data.shape
        n_params = n_components * (n_features + 1)

        # Compute the BIC score
        bic = -2 * log_likelihood + n_params * np.log(n_samples)
        bic_scores.append(bic)

    # Plot the BIC scores as a function of the number of components
    plt.plot(n_components_range, bic_scores, 'o-')
    plt.xlabel('Number of components')
    plt.ylabel('BIC score')
    plt.title(f'BIC elbow test for {decomposition_method} decomposition')
    plt.show()

    # Find the elbow point
    diff = np.diff(bic_scores)
    elbow_index = np.argmax(diff) + 1
    optimal_n_components = n_components_range[elbow_index]
    print(f'The optimal number of components is {optimal_n_components}')

    return optimal_n_components

In [34]:
bic_elbow(xrf_map, decomposition_method='sklearn_pca', n_components_range=np.arange(2, 11))

AttributeError: 'function' object has no attribute 'PCA'

In [29]:
with h5py.File(data, 'r') as h5:
    xrf_data = h5['/xrmmap/mcasum/counts'][:]

xrf_map = hs.signals.Signal1D(xrf_data)

optimal_n_components = bic_elbow(xrf_map, decomposition_method='PCA', n_components_range=np.arange(2, 20))

ValueError: Unknown decomposition method: PCA

## re-explore

get 6 good factors... need to label peaks etc but starting to pull out data.



In [120]:
xrf_stack.decomposition(normalize_poissonian_noise=False, algorithm='NMF', output_dimension=5)


xrf_stack.plot_decomposition_results()

/Users/joshuashea/xraylarch/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


Decomposition info:
  normalize_poissonian_noise=False
  algorithm=NMF
  output_dimension=5
  centre=None
scikit-learn estimator:
NMF(n_components=5)


/Users/joshuashea/xraylarch/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [ ]:
xrf_stack.decomposition(normalize_poissonian_noise=True, algorithm='NMF', output_dimension=5)


xrf_stack.plot_decomposition_results()

## examine with factor analysis

In [122]:
pipeline = Pipeline([("FA", FactorAnalysis(n_components=5,rotation = 'varimax'))])

xrf_stack.decomposition(normalize_poissonian_noise=False, algorithm=pipeline, return_info=True,output_dimension=11)

xrf_stack.plot_decomposition_results()

Decomposition info:
  normalize_poissonian_noise=False
  algorithm=Pipeline(steps=[('FA', FactorAnalysis(n_components=5, rotation='varimax'))])
  output_dimension=11
  centre=None
scikit-learn estimator:
Pipeline(steps=[('FA', FactorAnalysis(n_components=5, rotation='varimax'))])


In [214]:
xrf_stack.plot_decomposition_results()

In [123]:
fa_load= xrf_stack.get_decomposition_loadings()
#fa_load.save('snv_eds_analysis/plage_TD_eds_FA_4_load')

fa_fact= xrf_stack.get_decomposition_factors()
#fa_fact.set_elements(elements)


#fa_fact.save('snv_eds_analysis/plage_TD_eds_FA_4_fact')

In [ ]:
fa_facts, ydim,xdim=fa_load.data.shape

fact_load_vect= pd.DataFrame((fa_load.data.reshape(fa_facts, ydim*xdim).T), columns = ['Factor 1','Factor 2','Factor 3','Factor 4'])


In [126]:
fact_load_vect

NameError: name 'fact_load_vect' is not defined

In [ ]:
fa_facts, ydim,xdim=fa_load.data.shape
fa_vect=fa_load.data.reshape(fa_facts,ydim*xdim).transpose()
fa_vect.shape

In [ ]:
loadings_to_cluster=fa_vect[:,:fa_facts]
loadings_to_cluster.shape

In [38]:
class PGK(Probabilistic, GustafsonKesselMixin):
    pass

NameError: name 'Probabilistic' is not defined

In [ ]:
num_clus=bic
pgk = PGK(n_clusters =num_clus, n_init=10).fit(loadings_to_cluster)
# Process results for visualisation
print(pgk.memberships_)
labels_ = np.argmax(pgk.memberships_, axis=1)
memberships_ = pgk.memberships_[range(len(pgk.memberships_)), labels_] 

labels = labels_.reshape([ydim,xdim])
labels=hs.signals.Signal2D(labels)
labels.plot(cmap='tab10')

mems=pgk.memberships_.reshape(ydim,xdim,num_clus)
print(mems.shape)

mem_maps=hs.signals.Signal2D(mems)
mem_maps=mem_maps.transpose(signal_axes=(2,0))
mem_maps.change_dtype('float32')
mem_maps.plot(cmap='viridis')

In [ ]:
plt.close('all')

mem_maps.plot(cmap='viridis')

## next steps

1. makes sense to calcualte an elbow test or examine the BIC to determine the 'optimal' number of clusters
2. explore applying HDBSCAN on the FA/ PCA factors
3. would also to explore the UMAP pretreatment then HDBSCAN
4. maybe blind source separation (ie ICA) after PCA.

for 2 and 3 need a different virtual enviroment.